After data statistics check, index range 3173-13786 is selected to be converted into training data.

That is 

In [1]:
import pandas as pd
import numpy as np

rel_path = 'Daily_Precipitation_9412002_[GMT-12].csv'
prcpS1 = pd.read_csv(rel_path)

begin_date = prcpS1["Index"][3173]
end_date = prcpS1["Index"][13786]

print("from")
print(begin_date)
print("to")
print(end_date)



from
1987-09-09
to
2016-09-29


## Showing data

In [2]:
prcpS1_cleaned = prcpS1[3173:13787]
print(prcpS1_cleaned)

            Index     x
3173   1987-09-09   0.0
3174   1987-09-10   0.0
3175   1987-09-11   0.0
3176   1987-09-12  10.0
3177   1987-09-13   7.0
...           ...   ...
13782  2016-09-25   0.0
13783  2016-09-26   0.0
13784  2016-09-27   0.0
13785  2016-09-28   0.0
13786  2016-09-29   9.0

[10614 rows x 2 columns]


In [3]:
rel_path = 'Daily_Precipitation_9412003_[GMT-12].csv'

prcpS2 = pd.read_csv(rel_path)
prcpS2_cleaned = prcpS2[3173:13787]

print(prcpS2_cleaned)

            Index     x
3173   1987-09-09   4.3
3174   1987-09-10   0.0
3175   1987-09-11   0.0
3176   1987-09-12  19.0
3177   1987-09-13   2.0
...           ...   ...
13782  2016-09-25   0.0
13783  2016-09-26   0.0
13784  2016-09-27   0.0
13785  2016-09-28   0.0
13786  2016-09-29  12.9

[10614 rows x 2 columns]


In [4]:
rel_path = 'Daily_Precipitation_9414002_[GMT-12].csv'

prcpS3 = pd.read_csv(rel_path)
prcpS3_cleaned = prcpS3[3173:13787]

print(prcpS3_cleaned)

            Index     x
3173   1987-09-09  17.0
3174   1987-09-10   0.0
3175   1987-09-11   0.0
3176   1987-09-12  11.5
3177   1987-09-13   3.0
...           ...   ...
13782  2016-09-25   0.0
13783  2016-09-26   0.0
13784  2016-09-27   0.0
13785  2016-09-28   0.0
13786  2016-09-29  11.5

[10614 rows x 2 columns]


In [5]:
rel_path = 'Daily_Streamflow_TrancuraAntesDeLlafenco_9414001_[GMT-12].csv'

flowS1 = pd.read_csv(rel_path)
flowS1_cleaned = flowS1[3173:13787]

print(flowS1_cleaned)


            Index         x
3173   1987-09-09  133.0467
3174   1987-09-10  123.9533
3175   1987-09-11  117.6150
3176   1987-09-12  114.5050
3177   1987-09-13  124.3200
...           ...       ...
13782  2016-09-25   88.2996
13783  2016-09-26   88.6700
13784  2016-09-27   84.6946
13785  2016-09-28   83.4900
13786  2016-09-29   83.3871

[10614 rows x 2 columns]


In [6]:
rel_path = 'Daily_Streamflow_TrancuraEnCurarrehue_9412001_[GMT-12].csv'

flowS2 = pd.read_csv(rel_path)
flowS2_cleaned = flowS2[3173:13787]

print(flowS2_cleaned)

            Index        x
3173   1987-09-09  33.3400
3174   1987-09-10  32.2000
3175   1987-09-11  30.4900
3176   1987-09-12  29.5800
3177   1987-09-13  31.0600
...           ...      ...
13782  2016-09-25  27.7608
13783  2016-09-26  26.6908
13784  2016-09-27  26.0063
13785  2016-09-28  25.9046
13786  2016-09-29  25.6487

[10614 rows x 2 columns]


## Put them into one DataFrame

In [7]:
import math

def checkExistNan(data,curr_index,check_length):
    return_val = False
    for i in range(curr_index-check_length,curr_index):
        if math.isnan(data[i]):
            return_val = True
    return return_val


print(checkExistNan(flowS2["x"],4043,7))


train_data = []
for i in range(3173,13787):
    # train y would be flowS1 value, hence will check valid for past 3 days
    if math.isnan(flowS1["x"][i]):
        continue
    exist_nan_f1 = checkExistNan(flowS1["x"],i,3)
    exist_nan_f2 = checkExistNan(flowS2["x"],i,3)
    exist_nan_p1 = checkExistNan(prcpS1["x"],i,3)
    exist_nan_p2 = checkExistNan(prcpS2["x"],i,3)
    exist_nan_p3 = checkExistNan(prcpS3["x"],i,3)
    if exist_nan_f1 == True or \
        exist_nan_f2 == True or \
        exist_nan_p1 == True or \
        exist_nan_p2 == True or \
        exist_nan_p3 == True:
        continue
    #if no nan in this range exist in any dataset, add data to processed dataset
    curr_train_data = [flowS1["x"][i],
                  prcpS1["x"][i-1], prcpS2["x"][i-1], prcpS3["x"][i-1], flowS1["x"][i-1], flowS2["x"][i-1],
                  prcpS1["x"][i-2], prcpS2["x"][i-2], prcpS3["x"][i-2], flowS1["x"][i-2], flowS2["x"][i-2],
                  prcpS1["x"][i-3], prcpS2["x"][i-3], prcpS3["x"][i-3], flowS1["x"][i-3], flowS2["x"][i-3]]
    train_data.append(curr_train_data)
    
df_train_val = pd.DataFrame(train_data) 
df_train_val.columns = ["flowS1-curr",
                        "prcpS1-1day","prcpS2-1day","prcpS3-1day","flowS1-1day","flowS2-1day",
                        "prcpS1-2day","prcpS2-2day","prcpS3-2day","flowS1-2day","flowS2-2day",
                        "prcpS1-3day","prcpS2-3day","prcpS3-3day","flowS1-3day","flowS2-3day"]
print(df_train_val)



True
      flowS1-curr  prcpS1-1day  prcpS2-1day  prcpS3-1day  flowS1-1day  \
0        133.0467         12.0         17.0         23.5     139.7200   
1        123.9533          0.0          4.3         17.0     133.0467   
2        117.6150          0.0          0.0          0.0     123.9533   
3        114.5050          0.0          0.0          0.0     117.6150   
4        124.3200         10.0         19.0         11.5     114.5050   
...           ...          ...          ...          ...          ...   
9054      88.2996          0.0          0.0          0.0      88.5133   
9055      88.6700          0.0          0.0          0.0      88.2996   
9056      84.6946          0.0          0.0          0.0      88.6700   
9057      83.4900          0.0          0.0          0.0      84.6946   
9058      83.3871          0.0          0.0          0.0      83.4900   

      flowS2-1day  prcpS1-2day  prcpS2-2day  prcpS3-2day  flowS1-2day  \
0         33.9100         17.0         32.0  

## Turn value DF indo category DF

Data are turned from value to small, medium, large categories.
<br>
The mapping is as below:
<br>
<br>flowS1 small: val <= 81.665 medium: 81.665 < val < 135.5 large: val > 135.5
<br>
<br>flowS2 small: val <= 21.38 medium: 21.38 < val < 35.62 large: val > 35.62
<br>
<br>prcpS1 small: val = 0 medium: 0 < val < 7.0 large: val > 7.0
<br>
<br>prcpS2 small: val = 0 medium: 0 < val < 8.8 large: val > 8.8
<br>
<br>prcpS3 small: val = 0 medium: 0 < val < 10.0 large: val > 10.0



In [8]:
def valToCataMapping(val,val_type):
    return_str = ""
    match val_type:
        case "flowS1":
            if val <= 81.665:
                return_str = "small"
            elif val >= 135.5:
                return_str = "large"
            else:
                return_str = "medium"

        case "flowS2":
            if val <= 21.38:
                return_str = "small"
            elif val >= 35.63:
                return_str = "large"
            else:
                return_str = "medium"

        case "prcpS1":
            if val == 0:
                return_str = "small"
            elif val >= 7.0:
                return_str = "large"
            else:
                return_str = "medium"
        
        case "prcpS2":
            if val == 0:
                return_str = "small"
            elif val >= 8.8:
                return_str = "large"
            else:
                return_str = "medium"

        case "prcpS3":
            if val == 0:
                return_str = "small"
            elif val >= 10.0:
                return_str = "large"
            else:
                return_str = "medium"
    
        case _:
            print("not matched")
    return return_str        

train_data_cata = []
for i in range(len(df_train_val["flowS1-curr"])):
    curr_day_data = []
    data_curr = valToCataMapping(df_train_val["flowS1-curr"][i],"flowS1")
    curr_day_data.append(data_curr)

    data_curr = valToCataMapping(df_train_val["prcpS1-1day"][i],"prcpS1")
    curr_day_data.append(data_curr) 
    data_curr = valToCataMapping(df_train_val["prcpS2-1day"][i],"prcpS2") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["prcpS3-1day"][i],"prcpS3") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["flowS1-1day"][i],"flowS1") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["flowS2-1day"][i],"flowS2") 
    curr_day_data.append(data_curr)

    data_curr = valToCataMapping(df_train_val["prcpS1-2day"][i],"prcpS1") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["prcpS2-2day"][i],"prcpS2") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["prcpS3-2day"][i],"prcpS3") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["flowS1-2day"][i],"flowS1") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["flowS2-2day"][i],"flowS2") 
    curr_day_data.append(data_curr)

    data_curr = valToCataMapping(df_train_val["prcpS1-3day"][i],"prcpS1") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["prcpS2-3day"][i],"prcpS2") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["prcpS3-3day"][i],"prcpS3") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["flowS1-3day"][i],"flowS1") 
    curr_day_data.append(data_curr)
    data_curr = valToCataMapping(df_train_val["flowS2-3day"][i],"flowS2")     
    curr_day_data.append(data_curr)

    train_data_cata.append(curr_day_data)

train_cata_df = pd.DataFrame(train_data_cata)

train_cata_df.columns = ["flowS1-curr",
                        "prcpS1-1day","prcpS2-1day","prcpS3-1day","flowS1-1day","flowS2-1day",
                        "prcpS1-2day","prcpS2-2day","prcpS3-2day","flowS1-2day","flowS2-2day",
                        "prcpS1-3day","prcpS2-3day","prcpS3-3day","flowS1-3day","flowS2-3day",]

print(train_cata_df)
    



     flowS1-curr prcpS1-1day prcpS2-1day prcpS3-1day flowS1-1day flowS2-1day  \
0         medium       large       large       large       large      medium   
1         medium       small      medium       large      medium      medium   
2         medium       small       small       small      medium      medium   
3         medium       small       small       small      medium      medium   
4         medium       large       large       large      medium      medium   
...          ...         ...         ...         ...         ...         ...   
9054      medium       small       small       small      medium      medium   
9055      medium       small       small       small      medium      medium   
9056      medium       small       small       small      medium      medium   
9057      medium       small       small       small      medium      medium   
9058      medium       small       small       small      medium      medium   

     prcpS1-2day prcpS2-2day prcpS3-2da

In [9]:
train_cata_df.to_csv("bn_train_cata_data.csv")  
df_train_val.to_csv("bn_train_val_data.csv")  